# Parsel

Parsel is a library to extract data from HTML and XML using XPath and CSS selectors

---

## Usage

### Getting started

If you already know how to write CSS or XPath expressions, using Parsel is straightforward: you just need to create a Selector object for the HTML or XML text you want to parse, and use the available methods for selecting parts from the text and extracting data out of the result.

Creating a Selector object is simple:

In [1]:
from parsel import Selector

In [2]:
text = u"<html><body><h1>Hello, Parsel!</h1></body></html>"
sel = Selector(text=text)
sel

<Selector xpath=None data='<html><body><h1>Hello, Parsel!</h1></bod'>

One important thing to note is that if you’re using Python 2, make sure to use an unicode object for the text argument. Selector expects text to be an unicode object in Python 2 or an str object in Python 3.

Once you have created the Selector object, you can use CSS or XPath expressions to select elements:

In [3]:
sel.css('h1')

[<Selector xpath='descendant-or-self::h1' data='<h1>Hello, Parsel!</h1>'>]

In [4]:
sel.xpath('//h1')  # the same, but now with XPath

[<Selector xpath='//h1' data='<h1>Hello, Parsel!</h1>'>]

And extract data from those elements:

In [5]:
sel.css('h1::text').get()

'Hello, Parsel!'

In [6]:
sel.xpath('//h1/text()').getall()

['Hello, Parsel!']

XPath is a language for selecting nodes in XML documents, which can also be used with HTML. CSS is a language for applying styles to HTML documents. It defines selectors to associate those styles with specific HTML elements.

You can use either language you’re more comfortable with, though you may find that in some specific cases XPath is more powerful than CSS.

### Using selectors

To explain how to use the selectors we’ll use the requests library to download an example page located in the Parsel’s documentation:

[](http://parsel.readthedocs.org/en/latest/_static/selectors-sample1.html)


For the sake of completeness, here’s its full HTML code:

```
<html>
 <head>
  <base href='http://example.com/' />
  <title>Example website</title>
 </head>
 <body>
  <div id='images'>
   <a href='image1.html'>Name: My image 1 <br /><img src='image1_thumb.jpg' /></a>
   <a href='image2.html'>Name: My image 2 <br /><img src='image2_thumb.jpg' /></a>
   <a href='image3.html'>Name: My image 3 <br /><img src='image3_thumb.jpg' /></a>
   <a href='image4.html'>Name: My image 4 <br /><img src='image4_thumb.jpg' /></a>
   <a href='image5.html'>Name: My image 5 <br /><img src='image5_thumb.jpg' /></a>
  </div>
 </body>
</html>
```


So, let’s download that page and create a selector for it:

In [7]:
import requests
from parsel import Selector

In [8]:
url = 'http://parsel.readthedocs.org/en/latest/_static/selectors-sample1.html'
text = requests.get(url).text
selector = Selector(text=text)

Since we’re dealing with HTML, the default type for Selector, we don’t need to specify the type argument.

So, by looking at the HTML code of that page, let’s construct an XPath for selecting the text inside the title tag:

In [9]:
selector.xpath('//title/text()')

[<Selector xpath='//title/text()' data='Example website'>]

You can also ask the same thing using CSS instead:

In [10]:
selector.css('title::text')

[<Selector xpath='descendant-or-self::title/text()' data='Example website'>]

To actually extract the textual data, you must call the selector .get() or .getall() methods, as follows:

In [11]:
selector.xpath('//title/text()').getall()

['Example website']

In [12]:
selector.xpath('//title/text()').get()

'Example website'

.get() always returns a single result; if there are several matches, content of a first match is returned; if there are no matches, None is returned. .getall() returns a list with all results.

Notice that CSS selectors can select text or attribute nodes using CSS3 pseudo-elements:

In [13]:
selector.css('title::text').get()

'Example website'

As you can see, .xpath() and .css() methods return a SelectorList instance, which is a list of new selectors. This API can be used for quickly selecting nested data:

In [14]:
selector.css('img').xpath('@src').getall()

['image1_thumb.jpg',
 'image2_thumb.jpg',
 'image3_thumb.jpg',
 'image4_thumb.jpg',
 'image5_thumb.jpg']

If you want to extract only the first matched element, you can call the selector .get() (or its alias .extract_first() commonly used in previous parsel versions):

In [15]:
selector.xpath('//div[@id="images"]/a/text()').get()

'Name: My image 1 '

It returns None if no element was found:

In [16]:
selector.xpath('//div[@id="not-exists"]/text()').get() is None

True

Instead of using e.g. '@src' XPath it is possible to query for attributes using .attrib property of a Selector:

In [17]:
[img.attrib['src'] for img in selector.css('img')]

['image1_thumb.jpg',
 'image2_thumb.jpg',
 'image3_thumb.jpg',
 'image4_thumb.jpg',
 'image5_thumb.jpg']

As a shortcut, .attrib is also available on SelectorList directly; it returns attributes for the first matching element:

In [18]:
selector.css('img').attrib['src']

'image1_thumb.jpg'

This is most useful when only a single result is expected, e.g. when selecting by id, or selecting unique elements on a web page:

In [19]:
selector.css('base').attrib['href']

'http://example.com/'

Now we’re going to get the base URL and some image links:

In [20]:
selector.xpath('//base/@href').get()

'http://example.com/'

In [21]:
selector.css('base::attr(href)').get()

'http://example.com/'

In [22]:
selector.css('base').attrib['href']

'http://example.com/'

In [23]:
selector.xpath('//a[contains(@href, "image")]/@href').getall()

['image1.html', 'image2.html', 'image3.html', 'image4.html', 'image5.html']

In [24]:
selector.css('a[href*=image]::attr(href)').getall()

['image1.html', 'image2.html', 'image3.html', 'image4.html', 'image5.html']

In [25]:
selector.xpath('//a[contains(@href, "image")]/img/@src').getall()

['image1_thumb.jpg',
 'image2_thumb.jpg',
 'image3_thumb.jpg',
 'image4_thumb.jpg',
 'image5_thumb.jpg']

In [26]:
selector.css('a[href*=image] img::attr(src)').getall()

['image1_thumb.jpg',
 'image2_thumb.jpg',
 'image3_thumb.jpg',
 'image4_thumb.jpg',
 'image5_thumb.jpg']

### Nesting selectors

The selection methods (.xpath() or .css()) return a list of selectors of the same type, so you can call the selection methods for those selectors too. Here’s an example:

In [27]:
links = selector.xpath('//a[contains(@href, "image")]')
links.getall()

['<a href="image1.html">Name: My image 1 <br><img src="image1_thumb.jpg"></a>',
 '<a href="image2.html">Name: My image 2 <br><img src="image2_thumb.jpg"></a>',
 '<a href="image3.html">Name: My image 3 <br><img src="image3_thumb.jpg"></a>',
 '<a href="image4.html">Name: My image 4 <br><img src="image4_thumb.jpg"></a>',
 '<a href="image5.html">Name: My image 5 <br><img src="image5_thumb.jpg"></a>']

In [28]:
for index, link in enumerate(links):
    args = (index, link.xpath('@href').get(), link.xpath('img/@src').get())
    print('Link number %d points to url %r and image %r' % args)

Link number 0 points to url 'image1.html' and image 'image1_thumb.jpg'
Link number 1 points to url 'image2.html' and image 'image2_thumb.jpg'
Link number 2 points to url 'image3.html' and image 'image3_thumb.jpg'
Link number 3 points to url 'image4.html' and image 'image4_thumb.jpg'
Link number 4 points to url 'image5.html' and image 'image5_thumb.jpg'


### Selecting element attributes

There are several ways to get a value of an attribute. First, one can use XPath syntax:

In [29]:
selector.xpath("//a/@href").getall()

['image1.html', 'image2.html', 'image3.html', 'image4.html', 'image5.html']

XPath syntax has a few advantages: it is a standard XPath feature, and @attributes can be used in other parts of an XPath expression - e.g. it is possible to filter by attribute value.

parsel also provides an extension to CSS selectors (::attr(...)) which allows to get attribute values:

In [30]:
selector.css('a::attr(href)').getall()

['image1.html', 'image2.html', 'image3.html', 'image4.html', 'image5.html']

In addition to that, there is a .attrib property of Selector. You can use it if you prefer to lookup attributes in Python code, without using XPaths or CSS extensions:

In [31]:
[a.attrib['href'] for a in selector.css('a')]

['image1.html', 'image2.html', 'image3.html', 'image4.html', 'image5.html']

This property is also available on SelectorList; it returns a dictionary with attributes of a first matching element. It is convenient to use when a selector is expected to give a single result (e.g. when selecting by element ID, or when selecting an unique element on a page):

In [32]:
selector.css('base').attrib

{'href': 'http://example.com/'}

In [33]:
selector.css('base').attrib['href']

'http://example.com/'

.attrib property of an empty SelectorList is empty:

In [34]:
selector.css('foo').attrib

{}

### Using selectors with regular expressions

Selector also has a .re() method for extracting data using regular expressions. However, unlike using .xpath() or .css() methods, .re() returns a list of unicode strings. So you can’t construct nested .re() calls.

Here’s an example used to extract image names from the HTML code above:

In [35]:
selector.xpath('//a[contains(@href, "image")]/text()').re(r'Name:\s*(.*)')

['My image 1 ', 'My image 2 ', 'My image 3 ', 'My image 4 ', 'My image 5 ']

There’s an additional helper reciprocating .get() (and its alias .extract_first()) for .re(), named .re_first(). Use it to extract just the first matching string:

In [36]:
selector.xpath('//a[contains(@href, "image")]/text()').re_first(r'Name:\s*(.*)')

'My image 1 '

### Working with relative XPaths

Keep in mind that if you are nesting selectors and use an XPath that starts with /, that XPath will be absolute to the document and not relative to the selector you’re calling it from.

For example, suppose you want to extract all <p> elements inside <div> elements. First, you would get all <div> elements:

In [37]:
divs = selector.xpath('//div')

At first, you may be tempted to use the following approach, which is wrong, as it actually extracts all <p> elements from the document, not only those inside <div> elements:

In [38]:
for p in divs.xpath('//p'):  # this is wrong - gets all <p> from the whole document
    print(p.get())

This is the proper way to do it (note the dot prefixing the .//p XPath):

In [39]:
for p in divs.xpath('.//p'):  # extracts all <p> inside
    print(p.get())

Another common case would be to extract all direct <p> children:

In [40]:
for p in divs.xpath('p'):
    print(p.get())

For more details about relative XPaths see the Location Paths section in the XPath specification.

### Regular expressions
The test() function, for example, can prove quite useful when XPath’s starts-with() or contains() are not sufficient.

Example selecting links in list item with a “class” attribute ending with a digit:

In [41]:
from parsel import Selector

In [42]:
doc = u"""
<div>
    <ul>
        <li class="item-0"><a href="link1.html">first item</a></li>
        <li class="item-1"><a href="link2.html">second item</a></li>
        <li class="item-inactive"><a href="link3.html">third item</a></li>
        <li class="item-1"><a href="link4.html">fourth item</a></li>
        <li class="item-0"><a href="link5.html">fifth item</a></li>
    </ul>
</div>
"""

In [43]:
sel = Selector(text=doc)
sel.xpath('//li//@href').getall()

['link1.html', 'link2.html', 'link3.html', 'link4.html', 'link5.html']

In [44]:
sel.xpath('//li[re:test(@class, "item-\d$")]//@href').getall()

['link1.html', 'link2.html', 'link4.html', 'link5.html']

### Set operations
These can be handy for excluding parts of a document tree before extracting text elements for example.

Example extracting microdata (sample content taken from http://schema.org/Product) with groups of itemscopes and corresponding itemprops:

In [45]:
doc = u"""
<div itemscope itemtype="http://schema.org/Product">
 <span itemprop="name">Kenmore White 17" Microwave</span>
 <img src="kenmore-microwave-17in.jpg" alt='Kenmore 17" Microwave' />
 <div itemprop="aggregateRating"
   itemscope itemtype="http://schema.org/AggregateRating">
  Rated <span itemprop="ratingValue">3.5</span>/5
  based on <span itemprop="reviewCount">11</span> customer reviews
 </div>
 <div itemprop="offers" itemscope itemtype="http://schema.org/Offer">
   <span itemprop="price">$55.00</span>
   <link itemprop="availability" href="http://schema.org/InStock" />In stock
 </div>
 Product description:
 <span itemprop="description">0.7 cubic feet countertop microwave.
 Has six preset cooking categories and convenience features like
 Add-A-Minute and Child Lock.</span>
 Customer reviews:
 <div itemprop="review" itemscope itemtype="http://schema.org/Review">
   <span itemprop="name">Not a happy camper</span> -
   by <span itemprop="author">Ellie</span>,
   <meta itemprop="datePublished" content="2011-04-01">April 1, 2011
   <div itemprop="reviewRating" itemscope itemtype="http://schema.org/Rating">
     <meta itemprop="worstRating" content = "1">
     <span itemprop="ratingValue">1</span>/
     <span itemprop="bestRating">5</span>stars
   </div>
   <span itemprop="description">The lamp burned out and now I have to replace
   it. </span>
 </div>
 <div itemprop="review" itemscope itemtype="http://schema.org/Review">
   <span itemprop="name">Value purchase</span> -
   by <span itemprop="author">Lucas</span>,
   <meta itemprop="datePublished" content="2011-03-25">March 25, 2011
   <div itemprop="reviewRating" itemscope itemtype="http://schema.org/Rating">
     <meta itemprop="worstRating" content = "1"/>
     <span itemprop="ratingValue">4</span>/
     <span itemprop="bestRating">5</span>stars
   </div>
   <span itemprop="description">Great microwave for the price. It is small and
   fits in my apartment.</span>
 </div>
 ...
</div>
"""

In [46]:
sel = Selector(text=doc, type="html")
for scope in sel.xpath('//div[@itemscope]'):
    print("current scope:", scope.xpath('@itemtype').getall())
    props = scope.xpath('''
                set:difference(./descendant::*/@itemprop,
                               .//*[@itemscope]/*/@itemprop)''')
    print("    properties: %s" % (props.getall()))
    print("")

current scope: ['http://schema.org/Product']
    properties: ['name', 'aggregateRating', 'offers', 'description', 'review', 'review']

current scope: ['http://schema.org/AggregateRating']
    properties: ['ratingValue', 'reviewCount']

current scope: ['http://schema.org/Offer']
    properties: ['price', 'availability']

current scope: ['http://schema.org/Review']
    properties: ['name', 'author', 'datePublished', 'reviewRating', 'description']

current scope: ['http://schema.org/Rating']
    properties: ['worstRating', 'ratingValue', 'bestRating']

current scope: ['http://schema.org/Review']
    properties: ['name', 'author', 'datePublished', 'reviewRating', 'description']

current scope: ['http://schema.org/Rating']
    properties: ['worstRating', 'ratingValue', 'bestRating']



Here we first iterate over itemscope elements, and for each one, we look for all itemprops elements and exclude those that are themselves inside another itemscope.

### Other XPath extensions

Parsel also defines a sorely missed XPath extension function has-class that returns True for nodes that have all of the specified HTML classes:

In [47]:
from parsel import Selector

In [48]:
sel = Selector("""
        <p class="foo bar-baz">First</p>
        <p class="foo">Second</p>
        <p class="bar">Third</p>
        <p>Fourth</p>
""")

In [49]:
sel.xpath('//p[has-class("foo")]')

[<Selector xpath='//p[has-class("foo")]' data='<p class="foo bar-baz">First</p>'>,
 <Selector xpath='//p[has-class("foo")]' data='<p class="foo">Second</p>'>]

In [50]:
sel.xpath('//p[has-class("foo", "bar-baz")]')

[<Selector xpath='//p[has-class("foo", "bar-baz")]' data='<p class="foo bar-baz">First</p>'>]

In [51]:
sel.xpath('//p[has-class("foo", "bar")]')

[]

So XPath //p[has-class("foo", "bar-baz")] is roughly equivalent to CSS p.foo.bar-baz. Please note, that it is slower in most of the cases, because it’s a pure-Python function that’s invoked for every node in question whereas the CSS lookup is translated into XPath and thus runs more efficiently, so performance-wise its uses are limited to situations that are not easily described with CSS selectors.

Parsel also simplifies adding your own XPath extensions.

### Using text nodes in a condition

When you need to use the text content as argument to an XPath string function, avoid using .//text() and use just . instead.

This is because the expression .//text() yields a collection of text elements – a node-set. And when a node-set is converted to a string, which happens when it is passed as argument to a string function like contains() or starts-with(), it results in the text for the first element only.

Example:

In [52]:
from parsel import Selector

In [53]:
sel = Selector(text='<a href="#">Click here to go to the <strong>Next Page</strong></a>')

Converting a node-set to string:

In [54]:
sel.xpath('//a//text()').getall() # take a peek at the node-set

['Click here to go to the ', 'Next Page']

In [55]:
sel.xpath("string(//a[1]//text())").getall() # convert it to string

['Click here to go to the ']

A node converted to a string, however, puts together the text of itself plus of all its descendants:

In [56]:
sel.xpath("//a[1]").getall() # select the first node

['<a href="#">Click here to go to the <strong>Next Page</strong></a>']

In [57]:
sel.xpath("string(//a[1])").getall() # convert it to string

['Click here to go to the Next Page']

So, using the .//text() node-set won’t select anything in this case:

In [58]:
sel.xpath("//a[contains(.//text(), 'Next Page')]").getall()

[]

But using the . to mean the node, works:

In [59]:
sel.xpath("//a[contains(., 'Next Page')]").getall()

['<a href="#">Click here to go to the <strong>Next Page</strong></a>']

### Beware of the difference between //node[1] and (//node)[1]

`//node[1]` selects all the nodes occurring first under their respective parents.

`(//node)[1]` selects all the nodes in the document, and then gets only the first of them.

Example:

In [60]:
from parsel import Selector

In [61]:
sel = Selector(text="""
     <ul class="list">
         <li>1</li>
         <li>2</li>
         <li>3</li>
     </ul>
     <ul class="list">
         <li>4</li>
         <li>5</li>
         <li>6</li>
     </ul>""")

In [62]:
xp = lambda x: sel.xpath(x).getall()

This gets all first `<li>` elements under whatever it is its parent:

In [63]:
xp("//li[1]")

['<li>1</li>', '<li>4</li>']

And this gets the first `<li>` element in the whole document:

In [64]:
xp("(//li)[1]")

['<li>1</li>']

This gets all first `<li>` elements under an `<ul>` parent:

In [65]:
xp("//ul/li[1]")

['<li>1</li>', '<li>4</li>']

And this gets the first `<li>` element under an `<ul>` parent in the whole document:

In [66]:
xp("(//ul/li)[1]")

['<li>1</li>']

### When querying by class, consider using CSS

Because an element can contain multiple CSS classes, the XPath way to select elements by class is the rather verbose:

*[contains(concat(' ', normalize-space(@class), ' '), ' someclass ')]
If you use @class='someclass' you may end up missing elements that have other classes, and if you just use contains(@class, 'someclass') to make up for that you may end up with more elements that you want, if they have a different class name that shares the string someclass.

As it turns out, parsel selectors allow you to chain selectors, so most of the time you can just select by class using CSS and then switch to XPath when needed:

In [67]:
from parsel import Selector

In [68]:
sel = Selector(text='<div class="hero shout"><time datetime="2014-07-23 19:00">Special date</time></div>')
sel.css('.shout').xpath('./time/@datetime').getall()

['2014-07-23 19:00']

This is cleaner than using the verbose XPath trick shown above. Just remember to use the . in the XPath expressions that will follow.

### extract() and extract_first()

If you’re a long-time parsel (or Scrapy) user, you’re probably familiar with .extract() and .extract_first() selector methods. These methods are still supported by parsel, there are no plans to deprecate them.

However, parsel usage docs are now written using .get() and .getall() methods. We feel that these new methods result in more concise and readable code.

The following examples show how these methods map to each other.

SelectorList.get() is the same as SelectorList.extract_first():

In [69]:
selector.css('a::attr(href)').get()

'image1.html'

In [70]:
selector.css('a::attr(href)').extract_first()

'image1.html'

SelectorList.getall() is the same as SelectorList.extract():

In [71]:
selector.css('a::attr(href)').getall()

['image1.html', 'image2.html', 'image3.html', 'image4.html', 'image5.html']

In [72]:
selector.css('a::attr(href)').extract()

['image1.html', 'image2.html', 'image3.html', 'image4.html', 'image5.html']

Selector.get() is the same as Selector.extract():

In [73]:
selector.css('a::attr(href)')[0].get()

'image1.html'

In [74]:
selector.css('a::attr(href)')[0].extract()

'image1.html'

For consistency, there is also Selector.getall(), which returns a list:

In [75]:
selector.css('a::attr(href)')[0].getall()

['image1.html']

With the .extract() method it was not always obvious if a result is a list or not; to get a single result either .extract() or .extract_first() needed to be called, depending whether you had a Selector or SelectorList.

So, the main difference is that the outputs of .get() and .getall() are more predictable: .get() always returns a single result, .getall() always returns a list of all extracted results.